Import libraries for data cleaning 

In [1]:
import csv
from tokenize import String

from db_connection import get_db_engine
from db_connection import execute_query
import numpy as np
import pandas as pd
import seaborn as sns
import os
import re

Connection!


Reading CSV file

In [2]:
df_posters = pd.read_csv(r"C:\Users\aless\Desktop\Uni\Dataset\posters.csv\posters.csv")

Top records of CSV file: posters.csv

In [3]:
df_posters

,id,link
0,1000001,https://a.ltrbxd.com/resized/film-poster/2/7/7...
1,1000002,https://a.ltrbxd.com/resized/film-poster/4/2/6...
2,1000003,https://a.ltrbxd.com/resized/film-poster/4/7/4...
3,1000004,https://a.ltrbxd.com/resized/film-poster/5/1/5...
4,1000005,https://a.ltrbxd.com/resized/film-poster/2/4/0...
...,...,...
941592,1941593,NaN
941593,1941594,NaN
941594,1941595,https://a.ltrbxd.com/resized/film-poster/1/1/8...
941595,1941596,https://a.ltrbxd.com/resized/film-poster/1/1/8...


Checking for total dimension of the csv

In [4]:
df_posters.shape

(941597, 2)

Rename 'id' column in 'id_release'. "inplace = True" modifies directly the df without creating a copy

In [5]:
df_posters.rename(columns = {"id":"id_poster"} , inplace = True)


In [6]:
df_posters.columns

Index(['id_poster', 'link'], dtype='object')

Check info about column type

In [7]:
df_posters.dtypes

id_poster     int64
link         object
dtype: object

Correction of data type of the records

In [8]:
df_posters['id_poster'] = df_posters['id_poster'].astype('Int64')
df_posters['link'] = df_posters['link'].fillna(pd.NA).astype('string')

df_posters.dtypes

id_poster             Int64
link         string[python]
dtype: object

Check if there are duplicates

In [9]:
df_posters.duplicated().sum()


np.int64(0)

In [10]:
df_posters.drop_duplicates(inplace = True)
df_posters.duplicated().sum()

np.int64(0)

Check the missing values

In [11]:
df_posters.isnull().sum()

id_poster         0
link         180712
dtype: int64

Dropping records with missing value 

In [12]:
df_posters = df_posters.dropna()
df_posters.isnull().sum()

id_poster    0
link         0
dtype: int64

Connecting to PostgreSQL database and creating the table

In [13]:
engine = get_db_engine()
sql = \
    """CREATE TABLE IF NOT EXISTS posters (
            id_poster INTEGER CHECK (id_poster>=0),
            link TEXT CHECK (link<>''),

            PRIMARY KEY (id_poster, link)
            );
    """
execute_query(sql)

Success!


Population of the table using .to_sql function

In [14]:
df_posters.to_sql("posters", engine, if_exists="append", index=False)

885